# Importing Libraries


In [ ]:
import json
import os
import math
import librosa

# Mounting drive to fetch the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Feature Extraction

In [ ]:
DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/genres_original"
JSON_PATH = "data_10.json"
SAMPLE_RATE = 22050
TRACK_DURATION = 30 # measured in seconds
SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

num_mfcc=20
n_fft=2048
hop_length=512
num_segments=10

In [ ]:
data = {
    "mapping": [],
    "labels": [],
    "mfcc": []
}

In [ ]:
samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

In [ ]:
# loop through all genre sub-folder
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(DATASET_PATH)):

    if dirpath is not DATASET_PATH:

        semantic_label = dirpath.split("\\")[-1]
        data["mapping"].append(semantic_label)
        print("\nProcessing: {}".format(semantic_label))

        for f in filenames:

            file_path = os.path.join(dirpath, f)
            try:
                signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
            except:
                continue

            for d in range(num_segments):

                start = samples_per_segment * d
                finish = start + samples_per_segment

                # extract mfcc
                mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
                mfcc = mfcc.T

                # store only mfcc feature with expected number of vectors
                if len(mfcc) == num_mfcc_vectors_per_segment:
                    data["mfcc"].append(mfcc.tolist())
                    data["labels"].append(i-1)
                    print("{}, segment:{}".format(file_path, d+1))


with open("data_final.json", "w") as fp:
    json.dump(data, fp, indent=4)

Streaming output truncated to the last 5000 lines.
/content/drive/MyDrive/Colab Notebooks/genres_original/classical/classical.00049.wav, segment:10
/content/drive/MyDrive/Colab Notebooks/genres_original/classical/classical.00052.wav, segment:1
/content/drive/MyDrive/Colab Notebooks/genres_original/classical/classical.00052.wav, segment:2
/content/drive/MyDrive/Colab Notebooks/genres_original/classical/classical.00052.wav, segment:3
/content/drive/MyDrive/Colab Notebooks/genres_original/classical/classical.00052.wav, segment:4
/content/drive/MyDrive/Colab Notebooks/genres_original/classical/classical.00052.wav, segment:5
/content/drive/MyDrive/Colab Notebooks/genres_original/classical/classical.00052.wav, segment:6
/content/drive/MyDrive/Colab Notebooks/genres_original/classical/classical.00052.wav, segment:7
/content/drive/MyDrive/Colab Notebooks/genres_original/classical/classical.00052.wav, segment:8
/content/drive/MyDrive/Colab Notebooks/genres_original/classical/classical.00052.wav

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


/content/drive/MyDrive/Colab Notebooks/genres_original/jazz/jazz.00057.wav, segment:1
/content/drive/MyDrive/Colab Notebooks/genres_original/jazz/jazz.00057.wav, segment:2
/content/drive/MyDrive/Colab Notebooks/genres_original/jazz/jazz.00057.wav, segment:3
/content/drive/MyDrive/Colab Notebooks/genres_original/jazz/jazz.00057.wav, segment:4
/content/drive/MyDrive/Colab Notebooks/genres_original/jazz/jazz.00057.wav, segment:5
/content/drive/MyDrive/Colab Notebooks/genres_original/jazz/jazz.00057.wav, segment:6
/content/drive/MyDrive/Colab Notebooks/genres_original/jazz/jazz.00057.wav, segment:7
/content/drive/MyDrive/Colab Notebooks/genres_original/jazz/jazz.00057.wav, segment:8
/content/drive/MyDrive/Colab Notebooks/genres_original/jazz/jazz.00057.wav, segment:9
/content/drive/MyDrive/Colab Notebooks/genres_original/jazz/jazz.00057.wav, segment:10
/content/drive/MyDrive/Colab Notebooks/genres_original/jazz/jazz.00056.wav, segment:1
/content/drive/MyDrive/Colab Notebooks/genres_origina

In [ ]:
import json
with open("data_final.json", "r") as fp:
    data = json.load(fp)

In [ ]:
import numpy as np
x=np.array(data["mfcc"])
x1=np.array(data["mfcc"])

In [ ]:
y=np.array(data["labels"])
y1=np.array(data["labels"])

# Splitting dataset into training and testing set


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)
x_train_1,x_test_1,y_train_1,y_test_1 = train_test_split(x1,y1,test_size=0.3)

# Defining and Designing the Sequential model

In [ ]:
#Sequential

import tensorflow.keras as keras
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(x.shape[1],x.shape[2])),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dense(1024,activation='relu'),
    keras.layers.Dense(10,activation='softmax')
])

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.fit(
    x_train,y_train,batch_size=32,epochs=50
)

Epoch 1/50
219/219 [==============================] - 13s 56ms/step - loss: 8.9265 - accuracy: 0.3960
Epoch 2/50
219/219 [==============================] - 12s 56ms/step - loss: 1.3219 - accuracy: 0.5481
Epoch 3/50
219/219 [==============================] - 12s 56ms/step - loss: 1.1168 - accuracy: 0.6137
Epoch 4/50
219/219 [==============================] - 13s 60ms/step - loss: 1.0010 - accuracy: 0.6532
Epoch 5/50
219/219 [==============================] - 13s 58ms/step - loss: 1.1218 - accuracy: 0.6790
Epoch 6/50
219/219 [==============================] - 12s 54ms/step - loss: 0.8609 - accuracy: 0.7123
Epoch 7/50
219/219 [==============================] - 12s 55ms/step - loss: 0.6753 - accuracy: 0.7638
Epoch 8/50
219/219 [==============================] - 12s 56ms/step - loss: 0.6384 - accuracy: 0.7860
Epoch 9/50
219/219 [==============================] - 12s 55ms/step - loss: 0.5972 - accuracy: 0.7954
Epoch 10/50
219/219 [==============================] - 12s 56ms/step - loss: 0.597

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
model.evaluate(x_test,y_test,batch_size=32)

94/94 [==============================] - 1s 11ms/step - loss: 2.5620 - accuracy: 0.6953


[2.5619964599609375, 0.6952603459358215]

# Defining and Designing the CNN Model

In [ ]:
#CNN

import tensorflow.keras as keras
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model

model2 = Sequential()

model2.add(Conv1D(120,5, padding = 'same',input_shape=(x1.shape[1],x1.shape[2]),activation = 'relu'))
model2.add(MaxPooling1D(pool_size=(6), padding = 'same'))
model2.add(Conv1D(120,5, padding = 'same' ,activation = 'relu'))
model2.add(Dropout(0.2))
model2.add(Flatten())
model2.add(Dense(10, activation = 'softmax'))

In [ ]:
model2.compile(loss='sparse_categorical_crossentropy',
      optimizer='rmsprop',
      metrics=['accuracy'])

In [ ]:
model2.fit(x_train_1,y_train_1,batch_size = 32, epochs = 50, validation_data = (x_test_1, y_test_1))

In [ ]:
model2.evaluate(x_test_1,y_test_1,batch_size=32)

94/94 [==============================] - 1s 8ms/step - loss: 7.9023 - accuracy: 0.5981


[7.902261734008789, 0.5981308221817017]

# Saving the hierarchy file for the two models -> Sequential and CNN

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks/MusicModel_v3_model.h5")
model2.save("/content/drive/MyDrive/Colab Notebooks/MusicModel_v3_model_2.h5")

In [ ]:
signal, sample_rate = librosa.load("/content/drive/MyDrive/Colab Notebooks/genres_original/disco/disco.00031.wav", sr=SAMPLE_RATE)

In [ ]:
lst = []

for d in range(num_segments):
    start = samples_per_segment * d
    finish = start + samples_per_segment

    # extract mfcc
    mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    mfcc = mfcc.T

    # store only mfcc feature with expected number of vectors
    if len(mfcc) == num_mfcc_vectors_per_segment:
        lst.append(mfcc.tolist())
        print("{}, segment:{}".format("/content/drive/MyDrive/Colab Notebooks/genres_original/disco/disco.00031.wav", d+1))

/content/drive/MyDrive/Colab Notebooks/genres_original/disco/disco.00031.wav, segment:1
/content/drive/MyDrive/Colab Notebooks/genres_original/disco/disco.00031.wav, segment:2
/content/drive/MyDrive/Colab Notebooks/genres_original/disco/disco.00031.wav, segment:3
/content/drive/MyDrive/Colab Notebooks/genres_original/disco/disco.00031.wav, segment:4
/content/drive/MyDrive/Colab Notebooks/genres_original/disco/disco.00031.wav, segment:5
/content/drive/MyDrive/Colab Notebooks/genres_original/disco/disco.00031.wav, segment:6
/content/drive/MyDrive/Colab Notebooks/genres_original/disco/disco.00031.wav, segment:7
/content/drive/MyDrive/Colab Notebooks/genres_original/disco/disco.00031.wav, segment:8
/content/drive/MyDrive/Colab Notebooks/genres_original/disco/disco.00031.wav, segment:9
/content/drive/MyDrive/Colab Notebooks/genres_original/disco/disco.00031.wav, segment:10


In [ ]:
x=np.array(lst)
x1 = np.array(lst)

# Loading the two models for testing the output

In [ ]:
var = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/MusicModel_v3_model.h5")
var1 = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/MusicModel_v3_model_2.h5")

In [ ]:
final = np.argmax((var.predict(x) > 0.5).astype("int32"))
final_2 = np.argmax((var1.predict(x1) > 0.5).astype("int32"))
final_2 = final_2-(10*(final_2//10))

# Printing the output for a user input

In [ ]:
print("Using Linear model: Label of genre->", final)
print("Using CNN model: Label of genre->", final_2)

Using Linear model: Label of genre-> 3
Using CNN model: Label of genre-> 2


In [ ]:
if final == 0:
  print(" The predicted genre is classical")
elif final == 1:
  print("The predicted genre is blues")
elif final == 2:
  print("The predicted genre is reggae")
elif final == 3:
  print("The predicted genre is disco")
elif final == 4:
  print("The predicted genre is rock")
elif final == 5:
  print("The predicted genre is jazz")
elif final == 6:
  print("The predicted genre is country")
elif final == 7:
  print("The predicted genre is pop")
elif final == 8:
  print("The predicted genre is hiphop")
else:
  print("The predicted genre is metal")     

The predicted genre is disco


In [ ]:
if final_2 == 0:
  print(" The predicted genre is classical")
elif final_2 == 1:
  print("The predicted genre is blues")
elif final_2 == 2:
  print("The predicted genre is reggae")
elif final_2 == 3:
  print("The predicted genre is disco")
elif final_2 == 4:
  print("The predicted genre is rock")
elif final_2 == 5:
  print("The predicted genre is jazz")
elif final_2 == 6:
  print("The predicted genre is country")
elif final_2 == 7:
  print("The predicted genre is pop")
elif final_2 == 8:
  print("The predicted genre is hiphop")
else:
  print("The predicted genre is metal") 

The predicted genre is reggae
